<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202401/blob/main/Modulo07/Modulo07_PCA_Text_as_Data.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Ejemplos Principal Component Analysis

## Test Scores

In [ ]:
require("pacman")
p_load("tidyverse")

In [ ]:
dta<-read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/estudiantes.csv")
dta

In [ ]:
rownames(dta)<-dta$Estudiante
dta<-dta[,-1]
dta

In [ ]:
res_pca <- prcomp(dta)
res_pca

In [ ]:
p_load("factoextra")
eig_val <- get_eigenvalue(res_pca)
eig_val

In [ ]:
fviz_eig(res_pca, addlabels = TRUE, ylim = c(0, 70))

In [ ]:
ggplot(eig_val,aes(x=1:5,y=cumulative.variance.percent)) +
    geom_point()+
    geom_line() +
    theme_bw()

In [ ]:
fviz_pca_biplot(res_pca, 
                repel = TRUE,# Avoid text overlapping
                col.var = "#2E9FDF", # Variables color
                col.ind = "#696969"  # Individuals color
                )

##  Votos en el congreso

In [ ]:
votes <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/rollcall-votes.csv")
head(votes)

In [ ]:
legis <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/rollcall-members.csv")
head(legis)

In [ ]:
pcavote <- prcomp(votes, scale=TRUE)

fviz_eig(pcavote, addlabels = TRUE, ylim = c(0, 40))

In [ ]:
fviz_pca_biplot(pcavote)

In [ ]:
fviz_pca_biplot(pcavote, 
                col.ind = legis$party,
                palette = c("blue", "green", "red"),
                invisible ="var",
                repel=TRUE,
                labelsize = 2
                )

In [ ]:
votepc <- predict(pcavote) #
votepc[order(votepc[,1])[1:5],1]

In [ ]:
votepc[order(-votepc[,1])[1:5],1]

In [ ]:
votepc[order(votepc[,2])[1:5],2]

In [ ]:
votepc[order(-votepc[,2])[1:5],2]

In [ ]:
loadings <- pcavote$rotation[,1:2]

In [ ]:
hist(loadings[,1], main="", xlab="1st Principle Component Vote-Loadings",
     col=8, border=grey(.9))
abline(v=loadings[884,1], col=2)
text(x=loadings[884,1], y=550, "Afford. Health (amdt.)", xpd=TRUE, col=2, font=3)
abline(v=loadings[25,1], col=4)
text(x=loadings[25,1], y=550, "TARP", xpd=TRUE, col=4, font=3)

In [ ]:
loadings[order(abs(loadings[,2]), decreasing=TRUE)[1:5],2]

In [ ]:
sort(rowSums(votes==0), decreasing=TRUE)[1:5]

# Principal Component Regression

In [ ]:
shows <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/nbc_showdetails.csv", row.names=1)
shows$Genre <- factor(shows$Genre)
head(shows)

In [ ]:
plot(GRP ~ PE, data=shows, bg=c(4,2,3)[shows$Genre], pch=21, log="y")
legend("bottomright", legend=levels(shows$Genre), fill=c(4,2,3), bty="n")

In [ ]:
survey <- read.csv("https://github.com/ignaciomsarmiento/datasets/raw/main/nbc_pilotsurvey.csv", as.is=TRUE) 
survey$Show <- factor(survey$Show, levels=rownames(shows))
head(survey)

In [ ]:
Xpilot <- aggregate(survey[,-(1:2)],  ## -(1:2) to remove the variables 'show' and 'viewer' completely
                by=list(Show=survey$Show), mean)


rownames(Xpilot) <- Xpilot[,1]
Xpilot <- Xpilot[,-1]
head(Xpilot)

In [ ]:
PCApilot <- prcomp(Xpilot, scale=TRUE)

fviz_eig(PCApilot, addlabels = TRUE, ylim = c(0, 45))

In [ ]:
round(PCApilot$rotation[,1:3],1)

In [ ]:
fviz_pca_biplot(PCApilot, 
                col.ind = shows$Genre,
                palette = c("blue", "green", "red"),
                ylim=c(-3,3),
                xlim=c(-6,6), # hides "monarch cove",living with ed", and "next" but these are all tiny 
                invisible ="var",
                repel=TRUE,
                labelsize = 2
                )

In [ ]:
p_load("gamlr")

In [ ]:
zpilot <- predict(PCApilot)

PE <- shows$PE
zdf <- as.data.frame(zpilot)

summary(PEglm <- glm(PE ~ ., data=zdf[,1:2]))

In [ ]:
cvlasso <- cv.gamlr(x=as.matrix(Xpilot), y=PE, nfold=10)
coef(cvlasso) 

In [ ]:
cvlassoPCR <- cv.gamlr(x=zpilot, y=PE, nfold=10) 
coef(cvlassoPCR) 

In [ ]:
cvlassoboth <- cv.gamlr(x=as.matrix(cbind(Xpilot,zpilot)), y=PE, nfold=10)
coef(cvlassoboth)

In [ ]:
par(mfrow=c(1,3), mai=c(.2,.2,.5,.1), omi=c(.5,.5,0,0))
plot(cvlasso, main="Lasso on X", ylim=c(50,200), ylab="", xlab="", df=FALSE, bty="n")
plot(cvlassoPCR, main="Lasso on PCR", ylim=c(50,200), ylab="", xlab="", df=FALSE, bty="n")
plot(cvlassoboth, main="Lasso on X and PCR", ylim=c(50,200), ylab="", xlab="", df=FALSE, bty="n")
mtext(side=2, "mean squared error", outer=TRUE, line=2)
mtext(side=1, "log lamba", outer=TRUE, line=2)

# Texto como Datos: Ejemplos

## Regresión con Texto: Gentzkow and Shapiro

<div >
<img src = "figures/gentzgow_shapiro.png" />
</div>

In [ ]:
require("pacman")
p_load("tidyverse","textir")

In [ ]:
data("congress109", package = "textir")

In [ ]:
congress109Counts[c("Barack Obama","John Boehner"),995:998]

In [ ]:
congress109Ideology[c("Barack Obama","John Boehner"),1:5]

### Regresión 

$$
RepShare= X\beta  + u
$$

In [ ]:
dim(congress109Counts)

In [ ]:
X <- as(congress109Counts, "dMatrix")
repshare <- congress109Ideology$repshare

In [ ]:

X1 <- as.matrix(congress109Counts)
words<-colnames(X1)

covariance<-apply(X1,2,function(x) cor(x,repshare))

                  

wordcloud(words = words,
          freq = covariance,
          min.freq = 0,
          scale = c(1.5, 0.1), 
          max.words=200, 
          random.order=FALSE, 
          colors=brewer.pal(8, "YlOrRd"))


In [ ]:
# lasso
lassoslant <- cv.gamlr(X>0, repshare)  

B <- coef(lassoslant$gamlr)[-1,] 

In [ ]:
tail(sort(round(B[B!=0],4)),10)

In [ ]:
head(sort(round(B[B!=0],4)),10)

## Modelado de temas subyacentes


In [ ]:
data("we8there", package = "textir")

Waffle House #1258 in Bossier City, Louisiana:

*I normally would not revue a Waffle House but this one deserves it. The workers, Amanda, Amy, Cherry, James and J.D. were the most pleasant crew I have seen. While it was only lunch, B.L.T. and chili, it was great. The best thing was the 50’s rock and roll music, not to loud not to soft. This is a rare exception to what you all think a Waffle House is. Keep up the good work.*

*Overall: 5, Atmosphere: 5, Food: 5, Service: 5, Value: 5.*



Sartin’s Seafood in Nassau Bay, Texas,

*Had a very rude waitress and the manager wasn’t nice either.*
*Overall: 1, Atmosphere: 1, Food: 1, Service: 1, Value: 5.*

In [ ]:
x <- we8thereCounts
x[1,x[1,]!=0]

### PCA

In [ ]:
pca <- prcomp(x, scale=TRUE) # cuidado demora mucho

In [ ]:
tail(sort(pca$rotation[,1]))

In [ ]:
tail(sort(pca$rotation[,4]))

In [ ]:
v <- predict(pca)[,1:4]

In [ ]:
boxplot(v[,1] ~ we8thereRatings$Overall, xlab="overall rating", ylab="PC1 score")

### LDA

In [ ]:
p_load("maptpx") # para modelar topicos

x <- as.simple_triplet_matrix(we8thereCounts)

In [ ]:
tpc <- topics(x,K=10) 

In [ ]:
tpcs <- topics(x,K=5*(1:5), verb=1)

In [ ]:
summary(tpcs, n=10) 

In [ ]:
rownames(tpcs$theta)[order(tpcs$theta[,1], decreasing=TRUE)[1:10]]


In [ ]:
rownames(tpcs$theta)[order(tpcs$theta[,2], decreasing=TRUE)[1:10]]

In [ ]:
boxplot(tpcs$omega[,1] ~ we8thereRatings$Overall, col="gold", xlab="overall rating", ylab="topic 1 score")


In [ ]:
boxplot(tpcs$omega[,2] ~ we8thereRatings$Overall, col="pink", xlab="overall rating", ylab="topic 2 score")

In [ ]:
stars <- we8thereRatings[,"Overall"]

In [ ]:
Xtopics<-as(tpcs$omega, "dMatrix")

In [ ]:
p_load("gamlr")
regtopics.cv <- cv.gamlr(tpcs$omega, stars)

In [ ]:
regwords.cv <- cv.gamlr(we8thereCounts, stars)

In [ ]:
par(mfrow=c(1,2), mai=c(.3,.6,.7,.1), omi=c(.5,.2,0,0))
plot(regtopics.cv, ylim=c(1,2), xlab="", ylab="")
mtext("topic regression", font=2, line=2)
plot(regwords.cv, ylim=c(1,2), xlab="", ylab="")
mtext("token regression", font=2, line=2)
mtext(side=2, "mean squared error", outer=TRUE, line=0)
mtext(side=1, "log lamba", outer=TRUE, line=1)